In [38]:
from typing import Tuple, List

# Carregar o Léxico
Precisamos de uma lista de palavras válidas para a língua portuguesa - chamamos isso de léxico. Vamos utilizar um produzido por um time de especialistas da USP, no projeto [Unitex](http://www.nilc.icmc.usp.br/nilc/projects/unitex-pb/web/dicionarios.html).

Esse léxico possui mais de 75.000 palavras encontradas em textos em português. 

> ⚠ Uma análise do arquivo irá mostrar vários verbetes que não seriam válidos no term.ooo, como palavras em inglês (Windows ou wheel), pois esse léxico foi extraído de textos reais onde verbetes em outras línguas também são encontrados. Futuramente, poderíamos fazer uma redução dessa base para apenas palavras usuais.

Note que carregamos apenas palavras com 5 caracteres, uma vez que esse tamanho é fixo no `term.ooo`.

In [15]:
def load_dictionary(filename: str, word_length: int):
    """
    Loads a lexicon from a file, where the each word is the first token on a line.
    Tokens are separated by commas.
    """
    with open(filename, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.split(',')[0]
            line = line.strip()

            if len(line) == word_length:
                yield line


words = list(load_dictionary('DELAS_PB.dic', 5))

# Primeiro chute

Agora, vamos encontrar a palavra para ser nosso melhor primeiro "chute". Considerando uma distribuição de palavras que sigam o léxico (algo que não é, necessariamente, verdade ☹), vamos verificar quais são as letras mais frequentes. Dessa forma, vamos fazer o primeiro chute com uma palavra que contenha as letras mais frequentes na língua.

Note que as cinco letras mais frequentes podem não formar uma palavra válida. Por isto, vamos pegar a palavra do léxico que possua o maior número de letras frequentes.

Primeiro, nós contabilizamos as letras no léxico.

In [16]:
letters = {}

for word in words:
    for letter in word:
        if letter in letters:
            letters[letter] += 1
        else:
            letters[letter] = 1

# Sort by most common letter
sorted_letters = sorted(letters.items(), key=lambda x: x[1], reverse=True)
sorted_letters = list(map(lambda x: x[0], sorted_letters))


Agora, procuramos uma palavra no léxico que maximize as letras frequentes. Um ponto importante aqui é que não queremos repetições - i.e., não queremos uma palavra que tenha letras repetidas.

In [25]:
def find_word_with_letters(words: list, letters: list) -> str:
    """
    Finds the first word in the list that contains all the letters in the list.
    """
    for word in words:
        target = letters.copy()  # what letters are left to find

        for letter in word:
            if letter in target:
                target.remove(letter)

        if len(target) == len(letters) - 5:
            return word

    return None


def find_best_guess(words, ranked_letters):
    for size in range(5, len(ranked_letters)):
        letters = ranked_letters[:size]
        guess = find_word_with_letters(words, letters)

        if guess != None:
            return guess
        else:
            print("Could not find a word using the top letters {}".format(
                ''.join(letters)))


best_guess = find_best_guess(words, sorted_letters)
best_guess


Could not find a word using the top letters aorei


'ariel'

# Jogando o jogo

Com nosso primeiro chute em mãos, iniciamos o jogo. Precisamos de uma maneira para codificar a resposta que o jogo nos da para cada chute; utilizaremos uma string para isso, onde cada posição identifica o retorno do jogo, da seguinte maneira:

- Um espaço denota uma falha.
- Um caractere `i` indica que a letra está inclusa na palavra, mas não na posição correta.
- Um caracter `c` indica que a letra está inclusa na palavra e na posição correta.

Para jogar, informamos ao usuário qual deve ser o chute, e aguardamos que ele insira a resposta recebida. Aí computamos o melhor chute considerando o que já sabemos.

O melhor chute é aquele que obedece aquilo que já sabemos das palavras (por exemplo, quais letras devem estar presentes) e que prefere utilizar letras mais frequentes.

> Seguimos essa heurística por que não temos como deduzir a probabilidade de uma palavra aparecer no jogo, mas temos uma regra para determinar a probabilidade de uma palavra incluir uma letra. 

In [35]:
class Result:
	def __init__(self, result: str):
		if len(result) != 5:
			raise ValueError('Result must be 5 letters long')
		
		result = result.lower()

		if any(c not in ' ci' for c in result):
			raise ValueError('Result must be composed of spaces, "i" or "c"')

		self._result = result

	def __iter__(self):
		return enumerate(self._result)


In [39]:
class Guess:
	def __init__(self, guess: str) -> None:
		if len(guess) != 5:
			raise ValueError('Guess must be 5 letters long')

		guess = guess.lower()

		self._guess = guess

	def apply_result(self, result: Result) -> Tuple[List[str], str]:
		"""
		@returns A list of included letters, and the word know from the guess.
		"""
		included = []
		known = ''

		for i, r in result:
			if r == ' ':
				known += ' '
			elif r == 'c':
				known += self._guess[i]
			elif r == 'i':
				included.append(self._guess[i])

		return included, known

In [ ]:
def play_game(inital_guess: Guess, words: List[str], ranked_letters: List[str]):
    guesses = [inital_guess]

	# what letters should be in the word
    included_letters = []  
    
	# the word that is being guessed, every letter is blank until it is found
    final_word = '     '

    while True:
        try:
            result = input(
                f'Try the guess "{guesses[-1]}". What was the result? ')

            result = Result(result)

			

        except:
            print('Invalid input')
            continue

initial_guess = Guess(best_guess)
play_game(best_guess, words, sorted_letters)
